In [1]:
import numpy as np

In [2]:
%load_ext cython

# ABY2.0 Scalar Product

## Implementation

In [3]:
np.seterr(over='ignore') # Supress overflow warnings --> avoid warnings when modulo kicks in

def SP_Setup(l: int, dtype_Z2n: type = np.uint32, seed=42):
    # Set Ring info from dtype
    if not np.issubdtype(dtype_Z2n, np.integer):
        raise TypeError("dtype must be a numeric numpy type")
    Z2n_info = np.iinfo(dtype_Z2n)
    Z2n = (Z2n_info.min, Z2n_info.max)
    
    # Sample correlated input shares [δx] * [δy] = [δxy] 
    rng = np.random.default_rng(seed)
    δx0, δx1, δy0, δy1, δxy0 = \
        rng.integers(low=Z2n[0], high=Z2n[1], size=(5,l), dtype=dtype_Z2n)
    δxy1 = (δx0+δx1) * (δy0+δy1) - δxy0
    # Sample and output shares [δz]
    δz0, δz1 = rng.integers(low=Z2n[0], high=Z2n[1], size=(2), dtype=dtype_Z2n)
    return δx0, δx1, δy0, δy1, δxy0, δxy1, δz0, δz1

def SP_Online_local(j, Δx, δx_j, Δy, δy_j, δxy_j, δz_j):
    # Compute local share of [Δz]
    Δz_j = np.sum((Δx*Δy if j else 0) - Δx*δy_j - Δy*δx_j + δxy_j) + δz_j
    return Δz_j

def SP_Online(Δx, δx0, δx1, Δy, δy0, δy1, δxy0, δxy1, δz0, δz1):
    # P0
    Δz0 = SP_Online_local(0, Δx, δx0, Δy, δy0, δxy0, δz0)
    # P1
    Δz1 = SP_Online_local(1, Δx, δx1, Δy, δy1, δxy1, δz1)
    # Exchange shares and reconstruct common share Δy
    Δz = Δz0 + Δz1
    return Δz

def share(x, δx0, δx1):
    """Secret Share `x`, using the precomputed δx0 and δx1 to output the common Δx share"""
    return x+(δx0+δx1)

def reconstruct(Δx, δx0, δx1):
    """Reconstruct secret from triangular Secret Shares """
    return Δx-(δx0+δx1)

## Example

In [2]:
## SETUP
l         = 128
dtype_Z2n = np.uint32
δx0, δx1, δy0, δy1, δxy0, δxy1, δz0, δz1 = SP_Setup(l=128, dtype_Z2n=dtype_Z2n)

In [3]:
## ONLINE
# Data sharing
x = np.random.randint(0, 2**12, size=l, dtype=dtype_Z2n)
y = np.random.randint(0, 2**12, size=l, dtype=dtype_Z2n)
Δx = share(x, δx0, δx1)
Δy = share(y, δy0, δy1)

# SP computation
Δz = SP_Online(Δx, δx0, δx1, Δy, δy0, δy1, δxy0, δxy1, δz0, δz1)

In [4]:
# Reconstruct and open result
z = reconstruct(Δz, δz0, δz1)
z, x@y

(539056012, 539056012)

# Funshade

In [20]:
%%cython -c=-O3 -c=-maes -c=-msse -c=-msse2 -c=-DDTYPE_t=int64_t -c=-DUSE_LIBSODIUM -I fss -S fss/aes.c -S fss/fss.c -lsodium -c=-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
# cython: boundscheck=False, wraparound=False, initializedcheck=False, nonecheck=False, embedsignature=True
cimport numpy as np
import numpy as np
cimport cython

# from libc.string cimport memcpy, memset
from libcpp cimport bool

from libc.stdint cimport uint8_t
from libc.stdlib cimport malloc, free

cdef extern from "fss.h" nogil:
    ctypedef int DTYPE_t
    const size_t KEY_LEN
    DTYPE_t random_dtype()
    void random_buffer(uint8_t buffer[], size_t buffer_len)
    void SIGN_gen(DTYPE_t r_in, DTYPE_t r_out, uint8_t k0[KEY_LEN], uint8_t k1[KEY_LEN])
    DTYPE_t SIGN_eval(bool b, uint8_t kb[KEY_LEN], DTYPE_t x_hat)

# build the corresponding numpy dtype for DTYPE_t
cdef DTYPE_t tmp
DTYPE = np.asarray(<DTYPE_t[:1]>(&tmp)).dtype

cdef class funshade_key:
    cdef uint8_t* keystring
    def __cinit__(self):
        self.keystring = <uint8_t*> malloc(KEY_LEN*sizeof(uint8_t))
    def __init__(self):
        if not self.keystring:     raise MemoryError()
    def __dealloc__(self):
        if (self.keystring!=NULL): free(self.keystring)

    # ------------------------- PYTHON LEVEL PROPERTIES -------------------- #
    @property
    def array(self):
        return np.asarray(<uint8_t[:KEY_LEN]>self.keystring)
    @property
    def dtype(self):
        return DTYPE
    def __len__(self):
        return KEY_LEN
    def __repr__(self):
        return "<funshade key: 0x{}..., size={}B, ring={}>".format(
            self.array[:10].tobytes().hex(), len(self), self.dtype)
    
def funshade_setup(size_t l, DTYPE_t theta):
    # Beaver Triples for Pi-sharing scalar product
    cdef np.ndarray[DTYPE_t, ndim=1] delta_x_0 = np.empty(shape=(l), dtype=DTYPE),\
            delta_x_1 = np.empty(l, DTYPE), delta_y_0 = np.empty(l, DTYPE),\
            delta_y_1 = np.empty(l, DTYPE),delta_xy_0 = np.empty(l, DTYPE)
    random_buffer(<uint8_t*>delta_x_0.data,  l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_x_1.data,  l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_y_0.data,  l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_y_1.data,  l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_xy_0.data, l*sizeof(DTYPE_t))
    cdef np.ndarray[DTYPE_t, ndim=1] delta_xy_1 = \
            (delta_x_0+delta_x_1) * (delta_y_0+delta_y_1) - delta_xy_0
    
    # FSS interval containment
    cdef DTYPE_t r_0 = random_dtype()
    cdef DTYPE_t r_1 = random_dtype()
    cdef DTYPE_t r = r_0 + r_1
    cdef DTYPE_t r_theta_0 = r_0
    cdef DTYPE_t r_theta_1 = r_1 - theta
    
    cdef funshade_key k0 = funshade_key(), k1 = funshade_key()
    SIGN_gen(r, 0, &k0.keystring[0], &k1.keystring[0])
    
    return delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1


cpdef np.ndarray[DTYPE_t, ndim=1] funshade_share(DTYPE_t[:] v, DTYPE_t[:] delta_v):
    cdef size_t i, l = v.shape[0], 
    cdef np.ndarray[DTYPE_t, ndim=1] Delta_v = np.empty(shape=(l), dtype=DTYPE)
    # Compute local share of [Δz]
    for i in range(l):
        Delta_v[i] = delta_v[i] - v[i]
    return Delta_v

cpdef DTYPE_t funshade_eval_sp(bool j, DTYPE_t[:] Delta_x, DTYPE_t[:] Delta_y,
                         DTYPE_t[:] delta_x_j, DTYPE_t[:] delta_y_j, 
                         DTYPE_t[:] delta_xy_j, DTYPE_t delta_z_j):
    cdef size_t l = Delta_x.shape[0], i
    cdef DTYPE_t Delta_z_j = delta_z_j
    # Compute local share of [Δz]
    for i in range(l):
        Delta_z_j += j*Delta_x[i]*Delta_y[i] - Delta_x[i]*delta_y_j[i]\
                             + delta_xy_j[i] - Delta_y[i]*delta_x_j[i]
    return Delta_z_j

cpdef DTYPE_t funshade_eval_sign(bool j, funshade_key kj, DTYPE_t z_hat):
    return SIGN_eval(j, kj.keystring, z_hat)

In [38]:
delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1 = funshade_setup(128, 16)

In [39]:
l=128
x = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
y = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)

In [40]:
x@y

-91234443

In [41]:
Delta_x = funshade_share(x, delta_x_0+delta_x_1)
Delta_y = funshade_share(y, delta_y_0+delta_y_1)

In [42]:
r_theta_0, r_theta_1

(7334766522998866862, 8260493431446853755)

In [44]:
np.int32(r_theta_0)+np.int32(r_theta_1)

614243369

In [45]:
z0_hat = funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0)
z1_hat = funshade_eval_sp(1, Delta_x, Delta_y, delta_x_1, delta_y_1, delta_xy_1, r_theta_1)

In [48]:
z_hat = np.int32(z0_hat) +np.int32(z1_hat) 

In [49]:
np.int32(z_hat) - (np.int32(r_theta_0)+np.int32(r_theta_1))

-91234443

In [90]:
o0 = funshade_eval_sign(0, k0, z_hat)
o1 = funshade_eval_sign(1, k1, z_hat)

In [94]:
o1 + o0

0

## Timing tests

In [31]:
import timeit
timeit.timeit

In [42]:
n_times = 100000
for l in (128, 256, 512, 1024):
    t_setup = timeit.timeit(lambda: funshade_setup(l, 1), number=n_times)/n_times
    delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1 = funshade_setup(l, 1)
    x = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    y = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    Delta_x = funshade_share(x, delta_x_0+delta_x_1)
    Delta_y = funshade_share(y, delta_y_0+delta_y_1)
    t_share = timeit.timeit(lambda: funshade_share(x, delta_x_0+delta_x_1), number=n_times)/n_times
    z0_hat = funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0) 
    z1_hat = funshade_eval_sp(1, Delta_x, Delta_y, delta_x_1, delta_y_1, delta_xy_1, r_theta_1)
    t_eval_sp = timeit.timeit(lambda: funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0), number=n_times)/n_times
    z_hat = np.int32(z0_hat + z1_hat)
    o0 = funshade_eval_sign(0, k0, z_hat)
    o1 = funshade_eval_sign(1, k1, z_hat)
    t_eval_sign = timeit.timeit(lambda: funshade_eval_sign(0, k0, z_hat), number=n_times)/n_times
    print(f"{l},{t_setup},{t_share},{t_eval_sp},{t_eval_sign}")

128,3.9827146343886855e-05,1.8684956058859825e-06,1.349896714091301e-06,9.277286306023599e-06
256,6.0250639654695985e-05,2.0422036200761793e-06,1.5889262780547143e-06,9.272344075143337e-06
512,0.00010153724186122418,2.5422772020101545e-06,2.0289109274744986e-06,9.287703558802605e-06
1024,0.0001822995961084962,3.1342599913477896e-06,2.950666658580303e-06,9.273839145898819e-06


In [54]:
n_times = 100000
print(DTYPE)
for l in (128, 256, 512, 1024):
    t_setup = timeit.timeit(lambda: funshade_setup(l, 1), number=n_times)/n_times
    delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1 = funshade_setup(l, 1)
    x = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    y = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    Delta_x = funshade_share(x, delta_x_0+delta_x_1)
    Delta_y = funshade_share(y, delta_y_0+delta_y_1)
    t_share = timeit.timeit(lambda: funshade_share(x, delta_x_0+delta_x_1), number=n_times)/n_times
    z0_hat = funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0) 
    z1_hat = funshade_eval_sp(1, Delta_x, Delta_y, delta_x_1, delta_y_1, delta_xy_1, r_theta_1)
    t_eval_sp = timeit.timeit(lambda: funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0), number=n_times)/n_times
    z_hat = np.int16(z0_hat + z1_hat)
    o0 = funshade_eval_sign(0, k0, z_hat)
    o1 = funshade_eval_sign(1, k1, z_hat)
    t_eval_sign = timeit.timeit(lambda: funshade_eval_sign(0, k0, z_hat), number=n_times)/n_times
    print(f"{l},{t_setup},{t_share},{t_eval_sp},{t_eval_sign}")

int16
128,2.4804158210754395e-05,1.865949109196663e-06,1.4359067007899284e-06,4.698179811239243e-06
256,3.488733626902104e-05,1.9605836644768715e-06,1.6590922325849532e-06,4.682087861001491e-06
512,5.574137564748526e-05,2.277829237282276e-06,2.103770524263382e-06,4.691511951386929e-06
1024,9.664014294743538e-05,2.7539215981960295e-06,3.0082966387271883e-06,4.6931164711713795e-06


In [57]:
print(DTYPE)
for l in (128, 256, 512, 1024):
    t_setup = timeit.timeit(lambda: funshade_setup(l, 1), number=n_times)/n_times
    delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1 = funshade_setup(l, 1)
    x = np.random.randint(-2**5, 2**5, size=l, dtype=DTYPE)
    y = np.random.randint(-2**5, 2**5, size=l, dtype=DTYPE)
    Delta_x = funshade_share(x, delta_x_0+delta_x_1)
    Delta_y = funshade_share(y, delta_y_0+delta_y_1)
    t_share = timeit.timeit(lambda: funshade_share(x, delta_x_0+delta_x_1), number=n_times)/n_times
    z0_hat = funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0) 
    z1_hat = funshade_eval_sp(1, Delta_x, Delta_y, delta_x_1, delta_y_1, delta_xy_1, r_theta_1)
    t_eval_sp = timeit.timeit(lambda: funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0), number=n_times)/n_times
    z_hat = np.int8(z0_hat + z1_hat)
    o0 = funshade_eval_sign(0, k0, z_hat)
    o1 = funshade_eval_sign(1, k1, z_hat)
    t_eval_sign = timeit.timeit(lambda: funshade_eval_sign(0, k0, z_hat), number=n_times)/n_times
    print(f"{l},{t_setup},{t_share},{t_eval_sp},{t_eval_sign}")

int8
128,1.8951870352029802e-05,1.8030289933085442e-06,1.3066710159182548e-06,2.4274837598204612e-06
256,2.230082269757986e-05,1.932332888245583e-06,1.4977174252271652e-06,2.4340565875172613e-06
512,3.305837076157331e-05,2.199627533555031e-06,1.8789692595601083e-06,2.4422499909996987e-06
1024,5.3546695783734324e-05,2.6116602495312693e-06,2.6000332459807395e-06,2.446807660162449e-06


In [59]:
print(DTYPE)
for l in (128, 256, 512, 1024):
    t_setup = timeit.timeit(lambda: funshade_setup(l, 1), number=n_times)/n_times
    delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1 = funshade_setup(l, 1)
    x = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    y = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    Delta_x = funshade_share(x, delta_x_0+delta_x_1)
    Delta_y = funshade_share(y, delta_y_0+delta_y_1)
    t_share = timeit.timeit(lambda: funshade_share(x, delta_x_0+delta_x_1), number=n_times)/n_times
    z0_hat = funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0) 
    z1_hat = funshade_eval_sp(1, Delta_x, Delta_y, delta_x_1, delta_y_1, delta_xy_1, r_theta_1)
    t_eval_sp = timeit.timeit(lambda: funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0), number=n_times)/n_times
    z_hat = np.int64(z0_hat + z1_hat)
    o0 = funshade_eval_sign(0, k0, z_hat)
    o1 = funshade_eval_sign(1, k1, z_hat)
    t_eval_sign = timeit.timeit(lambda: funshade_eval_sign(0, k0, z_hat), number=n_times)/n_times
    print(f"{l},{t_setup},{t_share},{t_eval_sp},{t_eval_sign}")

int64
128,0.00012874264288693666,1.91250741481781e-06,1.3535232096910477e-06,2.493124309927225e-05
256,0.00011721820794045925,2.1982069686055183e-06,1.5952835232019424e-06,2.4897720329463482e-05
512,0.00019778695084154605,2.6394351944327356e-06,2.0489278808236124e-06,2.482534121721983e-05
1024,0.00036636827912181615,3.150496259331703e-06,3.0493321269750595e-06,2.486739069223404e-05


## Identification tests

In [17]:
%%cython -c=-O3 -c=-maes -c=-msse -c=-msse2 -c=-DDTYPE_t=int32_t -c=-DUSE_LIBSODIUM -I fss -S fss/aes.c -S fss/fss.c -lsodium -c=-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
# cython: boundscheck=False, wraparound=False, initializedcheck=False, nonecheck=False, embedsignature=True
cimport numpy as np
import numpy as np
cimport cython

# from libc.string cimport memcpy, memset
from libcpp cimport bool

from libc.stdint cimport uint8_t
from libc.stdlib cimport malloc, free

cdef extern from "fss.h" nogil:
    ctypedef int DTYPE_t
    const size_t KEY_LEN
    DTYPE_t random_dtype()
    void random_buffer(uint8_t buffer[], size_t buffer_len)
    void SIGN_gen(DTYPE_t r_in, DTYPE_t r_out, uint8_t k0[KEY_LEN], uint8_t k1[KEY_LEN])
    DTYPE_t SIGN_eval(bool b, uint8_t kb[KEY_LEN], DTYPE_t x_hat)

# build the corresponding numpy dtype for DTYPE_t
cdef DTYPE_t tmp
DTYPE = np.asarray(<DTYPE_t[:1]>(&tmp)).dtype

cdef class funshade_key:
    cdef uint8_t* keystring
    cdef size_t n_keys
    def __cinit__(self, size_t n_keys=1):
        assert n_keys>=1, "k must be a positive number"
        self.n_keys = n_keys
        self.keystring = <uint8_t*> malloc(n_keys*KEY_LEN*sizeof(uint8_t))
    def __init__(self, size_t n_keys=1):
        if not self.keystring:     raise MemoryError()
    def __dealloc__(self):
        if (self.keystring!=NULL): free(self.keystring)

    # ------------------------- PYTHON LEVEL PROPERTIES -------------------- #
    @property
    def array(self):
        return np.asarray(<uint8_t[:(self.n_keys*KEY_LEN)]>self.keystring)
    @property
    def dtype(self):
        return DTYPE
    def __len__(self):
        return KEY_LEN*self.n_keys
    def __repr__(self):
        return "<funshade key: 0x{}..., size={}B, ring={}>".format(
            self.array[:10].tobytes().hex(), len(self), self.dtype)
    
def funshade_setup(size_t l, size_t n_samples, DTYPE_t theta):
    # Beaver Triples for Pi-sharing scalar product
    cdef np.ndarray[DTYPE_t, ndim=1] delta_x_0 = np.empty(shape=(n_samples*l), dtype=DTYPE),\
            delta_x_1 = np.empty((n_samples*l), DTYPE), delta_y_0 = np.empty((n_samples*l), DTYPE),\
            delta_y_1 = np.empty((n_samples*l), DTYPE),delta_xy_0 = np.empty((n_samples*l), DTYPE)
    random_buffer(<uint8_t*>delta_x_0.data, n_samples*l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_x_1.data, n_samples*l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_y_0.data, n_samples*l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_y_1.data, n_samples*l*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>delta_xy_0.data, n_samples*l*sizeof(DTYPE_t))
    cdef np.ndarray[DTYPE_t, ndim=1] delta_xy_1 = \
            (delta_x_0+delta_x_1) * (delta_y_0+delta_y_1) - delta_xy_0
    
    # FSS interval containment
    cdef np.ndarray[DTYPE_t, ndim=1] r_0 = np.empty(shape=(n_samples), dtype=DTYPE)
    cdef np.ndarray[DTYPE_t, ndim=1] r_1 = np.empty(shape=(n_samples), dtype=DTYPE)
    random_buffer(<uint8_t*>r_0.data, n_samples*sizeof(DTYPE_t))
    random_buffer(<uint8_t*>r_1.data, n_samples*sizeof(DTYPE_t))
    cdef np.ndarray[DTYPE_t, ndim=1] r = r_0 + r_1
    # cdef DTYPE_t r_theta_0 = r_0
    cdef np.ndarray[DTYPE_t, ndim=1] r_theta_1 = r_1 - theta
    
    
    cdef funshade_key k0 = funshade_key(n_keys=n_samples), k1 = funshade_key(n_keys=n_samples)
    cdef size_t i
    for i in range(n_samples):
        SIGN_gen(r[i], 0, &k0.keystring[i*KEY_LEN], &k1.keystring[i*KEY_LEN])
    
    return delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_0, r_theta_1, k0, k1


cpdef np.ndarray[DTYPE_t, ndim=1] funshade_share_id(DTYPE_t[:] v, DTYPE_t[:] delta_v):
    cdef size_t i, l_v = v.shape[0], l = delta_v.shape[0], 
    cdef np.ndarray[DTYPE_t, ndim=1] Delta_v = np.empty(shape=(l), dtype=DTYPE)
    # Compute local share of [Δz]
    for i in range(l):
        Delta_v[i] = delta_v[i] - v[i%l_v]
    return Delta_v

cpdef DTYPE_t funshade_eval_sp_id(bool j, DTYPE_t[:] Delta_x, DTYPE_t[:] Delta_y,
                         DTYPE_t[:] delta_x_j, DTYPE_t[:] delta_y_j, 
                         DTYPE_t[:] delta_xy_j, DTYPE_t[:] delta_z_j):
    cdef size_t l = Delta_y.shape[0], n_samples = delta_z_j.shape[0], i
    cdef  np.ndarray[DTYPE_t, ndim=1] Delta_z = delta_z_j
    # Compute local share of [Δz]
    for i in range(l):
        Delta_z[i//n_samples] += j*Delta_x[i]*Delta_y[i] - Delta_x[i]*delta_y_j[i]\
                           + delta_xy_j[i] - Delta_y[i]*delta_x_j[i]
    return Delta_z

cpdef DTYPE_t funshade_eval_sign_id(bool j, funshade_key kj, DTYPE_t z_hat):
    return SIGN_eval(j, kj.keystring, z_hat)

In [16]:
funshade_setup(128, 1000, 12)

(array([ -299377626,   705646975,  -412055555, ...,  2057388159,
         1784328559, -2029459518], dtype=int32),
 array([ 1996979530,   598067285, -1345463155, ..., -1840846357,
          528066269,  -319301530], dtype=int32),
 array([  804437258,  -366834842, -1391689589, ..., -1226222077,
        -1863016688,   326125514], dtype=int32),
 array([ -469522801, -1502019620,   254534273, ...,  1300917337,
          948556520,   263600892], dtype=int32),
 array([-1367018111, -1218999724,  1615841848, ...,   833200001,
          672838885,  1819665354], dtype=int32),
 array([ 1543360111, -1984218540,   766396992, ...,   394785431,
          972841147, -1415378138], dtype=int32),
 array([-1696105060,  -907612242,  -694565519, -1632823259,  1179327115,
        -1858565132,   341285350, -1042816587,  1867417555,   789400221,
        -1196532737,  -240444355, -1429051645, -1591848541,  -962146675,
         -537332015,   801571808,  1105236659,  1537818966,  -498151131,
        -1154773976,  12

In [ ]:
print(DTYPE)
n_times= 100000
for l in (128, 256, 512, 1024):
    t_setup = timeit.timeit(lambda: funshade_setup(l, 1), number=n_times)/n_times
    delta_x_0, delta_x_1, delta_y_0, delta_y_1, delta_xy_0, delta_xy_1, r_theta_0, r_theta_1, k0, k1 = funshade_setup(l, 1)
    x = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    y = np.random.randint(-2**12, 2**12, size=l, dtype=DTYPE)
    Delta_x = funshade_share(x, delta_x_0+delta_x_1)
    Delta_y = funshade_share(y, delta_y_0+delta_y_1)
    t_share = timeit.timeit(lambda: funshade_share(x, delta_x_0+delta_x_1), number=n_times)/n_times
    z0_hat = funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0) 
    z1_hat = funshade_eval_sp(1, Delta_x, Delta_y, delta_x_1, delta_y_1, delta_xy_1, r_theta_1)
    t_eval_sp = timeit.timeit(lambda: funshade_eval_sp(0, Delta_x, Delta_y, delta_x_0, delta_y_0, delta_xy_0, r_theta_0), number=n_times)/n_times
    z_hat = np.int64(z0_hat + z1_hat)
    o0 = funshade_eval_sign(0, k0, z_hat)
    o1 = funshade_eval_sign(1, k1, z_hat)
    t_eval_sign = timeit.timeit(lambda: funshade_eval_sign(0, k0, z_hat), number=n_times)/n_times
    print(f"{l},{t_setup},{t_share},{t_eval_sp},{t_eval_sign}")

# Funshade: SP & FSS_sign

In [242]:
l         = 128
threshold = 0.4

rng = np.random.default_rng(seed=4242)
def sample_biometric_template():
    template = rng.exponential(size=l)   # choosing an arbitraty element distribution for the example.
    return template / np.linalg.norm(template)

# Biometric data generation
live_template = sample_biometric_template()
ref_template  = sample_biometric_template()

# Biometric data rescaling --> Fixed-point approximation
s = 2**12
live_template_s = list((live_template * s).astype(int))
ref_template_s  = list((ref_template  * s).astype(int))
threshold_s     = int(threshold * s**2)

In [35]:
seed = np.random.randint(2**8, size=16, dtype=np.uint8)

In [ ]:
%%cython -+ -c=/O2 -a
cimport numpy as np
import numpy as np
cimport cython

from libc.string cimport memset

np.import_array()


In [ ]:
Wrong! z=567957374    | z_hat=47143517     | r_in=-520813857 | theta=99294148 |z_hat_exp=-52150631 

In [91]:
theta=np.int32(99294148)

In [94]:
np.sum(r_in)+theta

-421519709

In [ ]:
47143517

In [101]:
x@y + np.sum(r_in)

47143517

In [74]:
x= np.int32([-45960336,-158798182,])
d_x= np.int32([-1276963481,780744389,])
d_x0= np.int32([1567945470,145062918,])
d_x1= np.int32([1450058345,635681471,])
D_x= np.int32([-1322923817,621946207,])
y= np.int32([-68414990,-17215093,])
d_y= np.int32([-1803887875,-290025441,])
d_y0= np.int32([64448122,1122102720,])
d_y1= np.int32([-1868335997,-1412128161,])
D_y= np.int32([-1872302865,-307240534,])
d_xy0= np.int32([195983585,1664694990,])
d_xy1= np.int32([-619879446,-2076100851,])
r_in= np.int32([-1542252855,1021438998,])

In [87]:
all(d_x0 + d_x1 ==d_x, ),\
all(d_y0 + d_y1 ==d_y),\
all(x +d_x0 + d_x1 ==D_x, ),\
all(y +d_y0 + d_y1 ==D_y, ),
all((d_y0 + d_y1)*(d_x0 + d_x1)-d_xy0 ==d_xy1, ),

(True,)

In [73]:
np.int32(-218081136) + (np.int32(2089589666)+np.int32(-2092059694))

-220551164

In [51]:
np.int32(-280353669*531415269 + -385494528*-469194163)

-1803088889